In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.manifold import TSNE
import umap
import time
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr
import math

In [5]:
all_datasets = catalog.list()
train_datasets = [name for name in all_datasets if name.endswith("_train")]
print(all_datasets)

['closing_prices', 'train_prices', 'val_prices', 'test_prices', 'windows_90_train', 'windows_90_val', 'windows_90_test', 'autoencoder_plots', 'pca_train', 'pca_val', 'pca_test', 'lle_train', 'lle_val', 'lle_test', 'umap_train', 'umap_val', 'umap_test', 'wavelet_train', 'wavelet_val', 'wavelet_test', 'fft_train', 'fft_val', 'fft_test', 'graph_train', 'graph_val', 'graph_test', 'tda_train', 'tda_val', 'tda_test', 'gaf_train', 'gaf_val', 'gaf_test', 'mtf_train', 'mtf_val', 'mtf_test', 'rp_train', 'rp_val', 'rp_test', 'gaf_clip_train', 'gaf_clip_val', 'gaf_clip_test', 'mtf_clip_train', 'mtf_clip_val', 'mtf_clip_test', 'rp_clip_train', 'rp_clip_val', 'rp_clip_test', 'gaf_dino_train', 'gaf_dino_val', 'gaf_dino_test', 'mtf_dino_train', 'mtf_dino_val', 'mtf_dino_test', 'rp_dino_train', 'rp_dino_val', 'rp_dino_test', 'gaf_resnet_train', 'gaf_resnet_val', 'gaf_resnet_test', 'mtf_resnet_train', 'mtf_resnet_val', 'mtf_resnet_test', 'rp_resnet_train', 'rp_resnet_val', 'rp_resnet_test', 'parameters'

In [21]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

def label_windows(series, window_size, threshold=0.01):
    """
    Labels each rolling window of length `window_size` in a 1D numeric array.
    Expects `series` to be a 1D array of floats.
    """
    arr = np.asarray(series, dtype=float)
    rets = (arr[window_size-1:] - arr[:-window_size+1]) / arr[:-window_size+1]
    labels = np.where(rets > threshold, 2,
             np.where(rets < -threshold, 0, 1))
    return labels.astype(int)


# 1) Load raw price‐DataFrames
train_df = catalog.load("train_prices")
val_df   = catalog.load("val_prices")
test_df  = catalog.load("test_prices")

# 2) Extract the numeric price series
window_size = 90
price_train = train_df["price_close"].astype(float).values
price_val   = val_df["price_close"].astype(float).values
price_test  = test_df["price_close"].astype(float).values

# 3) Compute labels
y_train = label_windows(price_train, window_size)
y_val   = label_windows(price_val,   window_size)
y_test  = label_windows(price_test,  window_size)

# 4) Discover which prefixes to evaluate (skip pure-image ones)
skip_images = {"gaf", "mtf", "rp"}
all_datasets   = catalog.list()
train_datasets = [n for n in all_datasets if n.endswith("_train")]
all_prefixes   = {ds.rsplit("_",1)[0] for ds in train_datasets}

embedding_prefixes = sorted(
    p for p in all_prefixes
    if not p.startswith(f"windows_{window_size}")
    and p not in skip_images
)

print("Will train/evaluate on:", embedding_prefixes)

# 5) Train & evaluate
for prefix in embedding_prefixes:
    print(f"\n=== {prefix} ===")
    raw_X_train = catalog.load(f"{prefix}_train")
    raw_X_test  = catalog.load(f"{prefix}_test")
    
    # coerce lists/DFs/Series into 2D np arrays
    X_train = np.asarray(raw_X_train, dtype=float)
    if X_train.ndim == 1:
        X_train = X_train.reshape(-1, 1)
    X_test  = np.asarray(raw_X_test,  dtype=float)
    if X_test.ndim  == 1:
        X_test  = X_test.reshape(-1, 1)
    
    clf = LogisticRegression(multi_class="multinomial", max_iter=1_000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(
        y_test, y_pred,
        target_names=["Bear","Neutral","Bull"]
    ))


[05/28/25 14:31:55] INFO     Loading data from train_prices (ParquetDataset)...                 ]8;id=277045;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=322030;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from val_prices (ParquetDataset)...                   ]8;id=922367;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=411389;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from test_prices (ParquetDataset)...                  ]8;id=946182;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=492269;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

Will train/evaluate on: ['fft', 'gaf_clip', 'gaf_dino', 'gaf_resnet', 'graph', 'lle', 'mtf_clip', 'mtf_dino', 'mtf_resnet', 'pca', 'rp_clip', 'rp_dino', 'rp_resnet', 'tda', 'umap', 'wavelet']

=== fft ===


                    INFO     Loading data from fft_train (ParquetDataset)...                    ]8;id=778153;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=14082;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

...

Accuracy: 0.7877618522601985
              precision    recall  f1-score   support

        Bear       0.66      0.68      0.67       439
     Neutral       0.28      0.26      0.27       116
        Bull       0.88      0.88      0.88      1259

    accuracy                           0.79      1814
   macro avg       0.60      0.60      0.60      1814
weighted avg       0.79      0.79      0.79      1814


=== gaf_resnet ===


                    INFO     Loading data from gaf_resnet_train (PickleDataset)...              ]8;id=385346;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=895918;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from gaf_resnet_test (PickleDataset)...               ]8;id=558359;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=685919;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    WARNING  /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/l ]8;id=995765;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=355429;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#109\109]8;;\
                             ib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247:                   
                             FutureWarning: 'multi_class' was deprecated in version 1.5 and will be                
                             removed in 1.7. From then on, it will always use 'multinomial'. Leave                 
                             it to its default value to avoid this warning.                                        
                               warnings.warn(                                                                      
                                                                                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:63                                                                                   │
│                                                                                                  │
│   60 │   │   X_test  = X_test.reshape(-1, 1)                                                     │
│   61 │                                                                                           │
│   62 │   clf = LogisticRegression(multi_class="multinomial", max_iter=1_000)                     │
│ ❱ 63 │   clf.fit(X_train, y_train)                                                               │
│   64 │   y_pred = clf.predict(X_test)                                                            │
│   65 │                                                                                           │
│   66 │   print("Accuracy:", accuracy_score(y_test, y_pred))                                      │
│                                                                                                  │
│ /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-package │
│ s/sklearn/base.py:1389 in wrapper                                                                │
│                                                                                                  │
│   1386 │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation               │
│   1387 │   │   │   │   )                                                                         │
│   1388 │   │   │   ):                                                                            │
│ ❱ 1389 │   │   │   │   return fit_method(estimator, *args, **kwargs)                             │
│   1390 │   │                                                                                     │
│   1391 │   │   return wrapper                                                                    │
│   1392                                                                                           │
│                                                                                                  │
│ /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-package │
│ s/sklearn/linear_model/_logistic.py:1350 in fit                                                  │
│                                                                                                  │
│   1347 │   │   else:                                                                             │
│   1348 │   │   │   n_threads = 1                                                                 │
│   1349 │   │                                                                                     │
│ ❱ 1350 │   │   fold_coefs_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, prefer=prefer)(  │
│   1351 │   │   │   path_func(                                                                    │
│   1352 │   │   │   │   X,                                                                        │
│   1353 │   │   │   │   y,                                                                        │
│                                                                                                  │
│ /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-package │
│ s/sklearn/utils/parallel.py:77 in __call__                                                       │
│                                                                                                  │
│    74 │   │   │   (_with_config(delayed_func, config), args, kwargs)                             │
│    75 │   │   │   for delayed_func, args, kwargs in iterable                                     │
│    76 │   │   )                                                                                  │
│ ❱  77 │   │   return super().__call__(iterable_with_config)

In [5]:
import numpy as np

# correct imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

def create_raw_windows(series, window_size):
    arr = np.asarray(series, dtype=float)
    return [arr[i:i+window_size] for i in range(len(arr) - window_size + 1)]

def realized_volatility(windows):
    return np.array([np.std(np.diff(np.log(w))) for w in windows])

# — PARAMETERS —
window_size = 90
skip_images = {"gaf", "mtf", "rp"}

# 1) Load train/test price series
train_df = catalog.load("train_prices")
test_df  = catalog.load("test_prices")
price_train = train_df["price_close"].astype(float).values
price_test  = test_df["price_close"].astype(float).values

# 2) Build windows & compute train thresholds
raw_wins_train = create_raw_windows(price_train, window_size)
vols_train     = realized_volatility(raw_wins_train)

thresh_25 = np.percentile(vols_train, 75)
y_binary_train = (vols_train >= thresh_25).astype(int)

low_thr, high_thr = np.percentile(vols_train, [33, 66])
y_multi_train = np.digitize(vols_train, bins=[low_thr, high_thr], right=True)

# 3) Apply thresholds on test set
raw_wins_test  = create_raw_windows(price_test, window_size)
vols_test      = realized_volatility(raw_wins_test)
y_binary_test  = (vols_test >= thresh_25).astype(int)
y_multi_test   = np.digitize(vols_test, bins=[low_thr, high_thr], right=True)

# 4) Discover embedding prefixes
all_ds   = catalog.list()
train_ds = [n for n in all_ds if n.endswith("_train")]
prefixes = sorted({
    ds.rsplit("_",1)[0] for ds in train_ds
    if not ds.startswith(f"windows_{window_size}") and ds.rsplit("_",1)[0] not in skip_images
})

# 5) Classifier pool
classifiers = {
    "LogisticRegression":   LogisticRegression(max_iter=500),
    "RandomForest":         RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting":     GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM (RBF kernel)":     SVC(kernel='rbf', probability=True, random_state=42),
    "K-Nearest Neighbors":  KNeighborsClassifier(n_neighbors=5),
}

def weighted_f1(clf, X_tr, y_tr, X_te, y_te):
    clf.fit(X_tr, y_tr)
    return f1_score(y_te, clf.predict(X_te), average="weighted")

# 6) Loop over embeddings → run both tasks
for prefix in prefixes:
    print(f"\n=== {prefix} ===")
    X_tr = np.asarray(catalog.load(f"{prefix}_train"), dtype=float)
    X_te = np.asarray(catalog.load(f"{prefix}_test"),  dtype=float)
    if X_tr.ndim == 1: X_tr = X_tr.reshape(-1,1)
    if X_te.ndim == 1: X_te = X_te.reshape(-1,1)
    
    print("  BINARY (low vs high vol):")
    for name, clf in classifiers.items():
        score = weighted_f1(clf, X_tr, y_binary_train, X_te, y_binary_test)
        print(f"    {name:25s} → wF1 = {score:.4f}")
    
    print("  MULTI-CLASS (low/med/high):")
    for name, clf in classifiers.items():
        if name == "LogisticRegression":
            clf = LogisticRegression(
                multi_class="multinomial", solver="lbfgs", max_iter=500
            )
        score = weighted_f1(clf, X_tr, y_multi_train, X_te, y_multi_test)
        print(f"    {name:25s} → wF1 = {score:.4f}")


[05/28/25 15:01:32] INFO     Loading data from train_prices (ParquetDataset)...                 ]8;id=374008;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=656913;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from test_prices (ParquetDataset)...                  ]8;id=617039;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=200030;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

Embedding prefixes: ['fft', 'gaf_clip', 'gaf_dino', 'gaf_resnet', 'graph', 'lle', 'mtf_clip', 'mtf_dino', 'mtf_resnet', 'pca', 'rp_clip', 'rp_dino', 'rp_resnet', 'tda', 'umap', 'wavelet']


[05/28/25 15:01:33] INFO     Loading data from fft_train (ParquetDataset)...                    ]8;id=706132;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=813304;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from fft_test (ParquetDataset)...                     ]8;id=34144;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=765958;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

...

                    INFO     Loading data from umap_train (ParquetDataset)...                   ]8;id=612297;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=534974;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from umap_test (ParquetDataset)...                    ]8;id=137667;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=56760;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

[05/28/25 15:01:49] INFO     Loading data from wavelet_train (ParquetDataset)...                ]8;id=365142;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=885574;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from wavelet_test (ParquetDataset)...                 ]8;id=603315;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=757101;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\


=== Ranking by avg weighted-F1 (BINARY) ===
 1. wavelet      → avg_wF1 = 0.6437
 2. pca          → avg_wF1 = 0.6301
 3. lle          → avg_wF1 = 0.5848
 4. gaf_clip     → avg_wF1 = 0.5712
 5. gaf_resnet   → avg_wF1 = 0.5562
 6. umap         → avg_wF1 = 0.5461
 7. rp_resnet    → avg_wF1 = 0.5372
 8. mtf_dino     → avg_wF1 = 0.5359
 9. gaf_dino     → avg_wF1 = 0.5287
10. rp_clip      → avg_wF1 = 0.5136
11. mtf_resnet   → avg_wF1 = 0.5089
12. mtf_clip     → avg_wF1 = 0.5041
13. fft          → avg_wF1 = 0.4945
14. rp_dino      → avg_wF1 = 0.4817
15. tda          → avg_wF1 = 0.4721
16. graph        → avg_wF1 = 0.4494

=== Ranking by avg weighted-F1 (MULTI-CLASS) ===
 1. pca          → avg_wF1 = 0.4347
 2. gaf_dino     → avg_wF1 = 0.4180
 3. gaf_resnet   → avg_wF1 = 0.4123
 4. gaf_clip     → avg_wF1 = 0.4103
 5. lle          → avg_wF1 = 0.4099
 6. rp_clip      → avg_wF1 = 0.4088
 7. graph        → avg_wF1 = 0.4028
 8. wavelet      → avg_wF1 = 0.3927
 9. tda          → avg_wF1 = 0.3879
10. m

In [2]:
import numpy as np

# correct imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

def create_raw_windows(series, window_size):
    arr = np.asarray(series, dtype=float)
    return [arr[i:i+window_size] for i in range(len(arr) - window_size + 1)]

def realized_volatility(windows):
    return np.array([np.std(np.diff(np.log(w))) for w in windows])

# — PARAMETERS —
window_size = 90
skip_images = {"gaf", "mtf", "rp"}

# 1) Load price series & build vol‐based labels
train_df = catalog.load("train_prices")
test_df  = catalog.load("test_prices")
price_train = train_df["price_close"].values.astype(float)
price_test  = test_df["price_close"].values.astype(float)

wins_tr = create_raw_windows(price_train, window_size)
wins_te = create_raw_windows(price_test,  window_size)
vols_tr = realized_volatility(wins_tr)
vols_te = realized_volatility(wins_te)

# binary
thresh_25       = np.percentile(vols_tr, 75)
y_binary_train  = (vols_tr >= thresh_25).astype(int)
y_binary_test   = (vols_te >=  thresh_25).astype(int)

# multi
low_thr, high_thr = np.percentile(vols_tr, [33, 66])
y_multi_train    = np.digitize(vols_tr, bins=[low_thr, high_thr], right=True)
y_multi_test     = np.digitize(vols_te, bins=[low_thr, high_thr], right=True)

# 2) Discover embeddings
all_ds   = catalog.list()
train_ds = [n for n in all_ds if n.endswith("_train")]
prefixes = sorted({
    ds.rsplit("_",1)[0] for ds in train_ds
    if not ds.startswith(f"windows_{window_size}") and ds.rsplit("_",1)[0] not in skip_images
})

# 3) Load & concatenate all embeddings into one big feature matrix
X_tr_parts = []
X_te_parts = []
for prefix in prefixes:
    arr_tr = np.asarray(catalog.load(f"{prefix}_train"), dtype=float)
    arr_te = np.asarray(catalog.load(f"{prefix}_test"),  dtype=float)
    if arr_tr.ndim == 1: arr_tr = arr_tr[:, None]
    if arr_te.ndim == 1: arr_te = arr_te[:, None]
    X_tr_parts.append(arr_tr)
    X_te_parts.append(arr_te)

X_tr_comb = np.hstack(X_tr_parts)
X_te_comb = np.hstack(X_te_parts)

# 4) Classifier pool
classifiers = {
    "LogisticRegression":   LogisticRegression(max_iter=500),
    "RandomForest":         RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting":     GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM (RBF kernel)":     SVC(kernel='rbf', probability=True, random_state=42),
    "K-Nearest Neighbors":  KNeighborsClassifier(n_neighbors=5),
}

def weighted_f1(clf, X_tr, y_tr, X_te, y_te):
    clf.fit(X_tr, y_tr)
    return f1_score(y_te, clf.predict(X_te), average="weighted")

# 5) Evaluate on the **combined** embedding
print("=== COMBINED EMBEDDING ===")

print("  BINARY (low vs high vol):")
for name, clf in classifiers.items():
    score = weighted_f1(clf, X_tr_comb, y_binary_train, X_te_comb, y_binary_test)
    print(f"    {name:25s} → wF1 = {score:.4f}")

print("\n  MULTI-CLASS (low/med/high):")
for name, clf in classifiers.items():
    if name == "LogisticRegression":
        clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=500)
    score = weighted_f1(clf, X_tr_comb, y_multi_train, X_te_comb, y_multi_test)
    print(f"    {name:25s} → wF1 = {score:.4f}")


[05/28/25 14:47:26] INFO     Loading data from train_prices (ParquetDataset)...                 ]8;id=664888;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=919514;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from test_prices (ParquetDataset)...                  ]8;id=986081;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=479771;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from fft_train (ParquetDataset)...                    ]8;id=758979;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=881961;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from fft_test (ParquetDataset)...                     ]8;id=450635;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=444463;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from gaf_clip_train (PickleDataset)...                ]8;id=463097;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=423102;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

...

                    INFO     Loading data from umap_train (ParquetDataset)...                   ]8;id=740696;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=210133;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from umap_test (ParquetDataset)...                    ]8;id=981142;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=685210;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from wavelet_train (ParquetDataset)...                ]8;id=568443;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=680762;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from wavelet_test (ParquetDataset)...                 ]8;id=462414;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=167675;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

=== COMBINED EMBEDDING ===
  BINARY (low vs high vol):
    K-Nearest Neighbors       → wF1 = 0.5340

  MULTI-CLASS (low/med/high):
    K-Nearest Neighbors       → wF1 = 0.4018


In [3]:
import numpy as np

# correct imports
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

def create_raw_windows(series, window_size):
    arr = np.asarray(series, dtype=float)
    return [arr[i:i+window_size] for i in range(len(arr) - window_size + 1)]

def realized_volatility(windows):
    return np.array([np.std(np.diff(np.log(w))) for w in windows])

# — PARAMETERS —
window_size = 90
skip_images = {"gaf", "mtf", "rp"}

# *** HERE: pick which embeddings to combine ***
# Leave as [] (or None) to use *all* available non-image embeddings:
selected_prefixes = ["pca", "umap", "wavelet"]  # e.g. only PCA, UMAP & Wavelet
# selected_prefixes = []  # <-- uncomment to combine *all* automatically discovered ones

# 1) Load price series & build vol‐based labels
train_df = catalog.load("train_prices")
test_df  = catalog.load("test_prices")
price_train = train_df["price_close"].values.astype(float)
price_test  = test_df["price_close"].values.astype(float)

wins_tr = create_raw_windows(price_train, window_size)
wins_te = create_raw_windows(price_test,  window_size)
vols_tr = realized_volatility(wins_tr)
vols_te = realized_volatility(wins_te)

# binary labels
thresh_25       = np.percentile(vols_tr, 75)
y_binary_train  = (vols_tr >= thresh_25).astype(int)
y_binary_test   = (vols_te >=  thresh_25).astype(int)

# multi-class labels
low_thr, high_thr   = np.percentile(vols_tr, [33, 66])
y_multi_train       = np.digitize(vols_tr, bins=[low_thr, high_thr], right=True)
y_multi_test        = np.digitize(vols_te, bins=[low_thr, high_thr], right=True)

# 2) Discover all non-image embeddings
all_ds    = catalog.list()
train_ds  = [n for n in all_ds if n.endswith("_train")]
all_pref  = sorted({
    ds.rsplit("_",1)[0] for ds in train_ds
    if not ds.startswith(f"windows_{window_size}") and ds.rsplit("_",1)[0] not in skip_images
})

# if user didn't pick any, combine all
if not selected_prefixes:
    prefixes = all_pref
else:
    prefixes = [p for p in selected_prefixes if p in all_pref]
    missing = set(selected_prefixes) - set(prefixes)
    if missing:
        raise ValueError(f"These embeddings not found: {missing}")

print(">>> combining embeddings:", prefixes)

# 3) Load & concatenate
X_tr_parts = []
X_te_parts = []
for prefix in prefixes:
    arr_tr = np.asarray(catalog.load(f"{prefix}_train"), dtype=float)
    arr_te = np.asarray(catalog.load(f"{prefix}_test"),  dtype=float)
    if arr_tr.ndim == 1: arr_tr = arr_tr[:, None]
    if arr_te.ndim == 1: arr_te = arr_te[:, None]
    X_tr_parts.append(arr_tr)
    X_te_parts.append(arr_te)

X_tr_comb = np.hstack(X_tr_parts)
X_te_comb = np.hstack(X_te_parts)

# 4) Classifier pool
classifiers = {
    "LogisticRegression":   LogisticRegression(max_iter=500),
    "RandomForest":         RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting":     GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM (RBF kernel)":     SVC(kernel='rbf', probability=True, random_state=42),
    "K-Nearest Neighbors":  KNeighborsClassifier(n_neighbors=5),
}

def weighted_f1(clf, X_tr, y_tr, X_te, y_te):
    clf.fit(X_tr, y_tr)
    return f1_score(y_te, clf.predict(X_te), average="weighted")

# 5) Evaluate on the **combined** embedding
print("\n=== COMBINED EMBEDDING ===")

print("  BINARY (low vs high vol):")
for name, clf in classifiers.items():
    score = weighted_f1(clf, X_tr_comb, y_binary_train, X_te_comb, y_binary_test)
    print(f"    {name:25s} → wF1 = {score:.4f}")

print("\n  MULTI-CLASS (low/med/high):")
for name, clf in classifiers.items():
    if name == "LogisticRegression":
        clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=500)
    score = weighted_f1(clf, X_tr_comb, y_multi_train, X_te_comb, y_multi_test)
    print(f"    {name:25s} → wF1 = {score:.4f}")


[05/28/25 14:48:44] INFO     Loading data from train_prices (ParquetDataset)...                 ]8;id=216821;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=535658;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from test_prices (ParquetDataset)...                  ]8;id=991695;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=877653;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

>>> combining embeddings: ['pca', 'umap', 'wavelet']


[05/28/25 14:48:45] INFO     Loading data from pca_train (ParquetDataset)...                    ]8;id=242071;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=242442;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from pca_test (ParquetDataset)...                     ]8;id=770894;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=45173;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

...

                    INFO     Loading data from wavelet_test (ParquetDataset)...                 ]8;id=794136;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=53901;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\


=== COMBINED EMBEDDING ===
  BINARY (low vs high vol):
    LogisticRegression        → wF1 = 0.4477
    RandomForest              → wF1 = 0.5148
    GradientBoosting          → wF1 = 0.4750
    SVM (RBF kernel)          → wF1 = 0.4514
    K-Nearest Neighbors       → wF1 = 0.6323

  MULTI-CLASS (low/med/high):


[05/28/25 14:50:50] WARNING  /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/l ]8;id=669196;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=797060;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#109\109]8;;\
                             ib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247:                   
                             FutureWarning: 'multi_class' was deprecated in version 1.5 and will be                
                             removed in 1.7. From then on, it will always use 'multinomial'. Leave                 
                             it to its default value to avoid this warning.                                        
                               warnings.warn(                                                                      
                                                                                                                   

    LogisticRegression        → wF1 = 0.4354


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:105                                                                                  │
│                                                                                                  │
│   102 for name, clf in classifiers.items():                                                      │
│   103 │   if name == "LogisticRegression":                                                       │
│   104 │   │   clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=500   │
│ ❱ 105 │   score = weighted_f1(clf, X_tr_comb, y_multi_train, X_te_comb, y_multi_test)            │
│   106 │   print(f"    {name:25s} → wF1 = {score:.4f}")                                           │
│   107                                                                                            │
│                                                                                                  │
│ in weighted_f1:90                                                                                │
│                                                                                                  │
│    87 }                                                                                          │
│    88                                                                                            │
│    89 def weighted_f1(clf, X_tr, y_tr, X_te, y_te):                                              │
│ ❱  90 │   clf.fit(X_tr, y_tr)                                                                    │
│    91 │   return f1_score(y_te, clf.predict(X_te), average="weighted")                           │
│    92                                                                                            │
│    93 # 5) Evaluate on the **combined** embedding                                                │
│                                                                                                  │
│ /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-package │
│ s/sklearn/base.py:1389 in wrapper                                                                │
│                                                                                                  │
│   1386 │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation               │
│   1387 │   │   │   │   )                                                                         │
│   1388 │   │   │   ):                                                                            │
│ ❱ 1389 │   │   │   │   return fit_method(estimator, *args, **kwargs)                             │
│   1390 │   │                                                                                     │
│   1391 │   │   return wrapper                                                                    │
│   1392                                                                                           │
│                                                                                                  │
│ /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-package │
│ s/sklearn/ensemble/_forest.py:487 in fit                                                         │
│                                                                                                  │
│    484 │   │   │   # that case. However, for joblib 0.12+ we respect any                         │
│    485 │   │   │   # parallel_backend contexts set at a higher level,                            │
│    486 │   │   │   # since correctness does not rely on using threads.                           │
│ ❱  487 │   │   │   trees = Parallel(                                                             │
│    488 │   │   │   │   n_jobs=self.n_jobs,                                                       │
│    489 │   │   │   │   verbose=self.verbose,               

In [4]:
import numpy as np
import itertools

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# 1) helper functions
def create_raw_windows(series, window_size):
    arr = np.asarray(series, dtype=float)
    return [arr[i:i+window_size] for i in range(len(arr) - window_size + 1)]

def realized_volatility(windows):
    return np.array([np.std(np.diff(np.log(w))) for w in windows])

def weighted_f1(clf, X_tr, y_tr, X_te, y_te):
    clf.fit(X_tr, y_tr)
    return f1_score(y_te, clf.predict(X_te), average="weighted")


# — PARAMETERS —
window_size   = 90
skip_images   = {"gaf", "mtf", "rp"}

# 2) build your vol-based labels
train_df      = catalog.load("train_prices")
test_df       = catalog.load("test_prices")
price_tr      = train_df["price_close"].astype(float).values
price_te      = test_df ["price_close"].astype(float).values

wins_tr       = create_raw_windows(price_tr, window_size)
wins_te       = create_raw_windows(price_te, window_size)
vols_tr       = realized_volatility(wins_tr)
vols_te       = realized_volatility(wins_te)

# 2a) binary: top 25% vs rest
thr25         = np.percentile(vols_tr, 75)
y_bin_tr      = (vols_tr >= thr25).astype(int)
y_bin_te      = (vols_te >= thr25).astype(int)

# 2b) multi: low/med/high via 33/66
low33, high66 = np.percentile(vols_tr, [33, 66])
y_mul_tr      = np.digitize(vols_tr, bins=[low33, high66], right=True)
y_mul_te      = np.digitize(vols_te, bins=[low33, high66], right=True)


# 3) discover your embedding prefixes
all_ds   = catalog.list()
train_ds = [n for n in all_ds if n.endswith("_train")]
prefixes = sorted({
    ds.rsplit("_",1)[0]
    for ds in train_ds
    if not ds.startswith(f"windows_{window_size}") 
       and ds.rsplit("_",1)[0] not in skip_images
})


# 4) prepare classifier constructors
clf_constructors = {
    # "LogisticRegression":   lambda multiclass=False: LogisticRegression(
    #                             **({"multi_class":"multinomial","solver":"lbfgs"} if multiclass else {}),
    #                             max_iter=500
    #                          ),
    # "RandomForest":         lambda multiclass=False: RandomForestClassifier(n_estimators=100, random_state=42),
    # "GradientBoosting":     lambda multiclass=False: GradientBoostingClassifier(n_estimators=100, random_state=42),
    # "SVM (RBF kernel)":     lambda multiclass=False: SVC(kernel='rbf', probability=True, random_state=42),
    "K-Nearest Neighbors":  lambda multiclass=False: KNeighborsClassifier(n_neighbors=5),
}


# 5) loop over all 2-element combinations & evaluate
results_bin = []
results_mul = []

for combo in itertools.combinations(prefixes, 2):
    # load + concat embeddings for this pair
    parts_tr = []
    parts_te = []
    for p in combo:
        arr_tr = np.asarray(catalog.load(f"{p}_train"), dtype=float)
        arr_te = np.asarray(catalog.load(f"{p}_test"),  dtype=float)
        if arr_tr.ndim == 1: arr_tr = arr_tr[:,None]
        if arr_te.ndim == 1: arr_te = arr_te[:,None]
        parts_tr.append(arr_tr)
        parts_te.append(arr_te)
    X_tr = np.hstack(parts_tr)
    X_te = np.hstack(parts_te)
    
    # binary task
    f1s = []
    for name, make_clf in clf_constructors.items():
        f1s.append(weighted_f1(make_clf(False), X_tr, y_bin_tr, X_te, y_bin_te))
    results_bin.append((combo, np.mean(f1s)))
    
    # multi-class task
    f1s = []
    for name, make_clf in clf_constructors.items():
        f1s.append(weighted_f1(make_clf(True), X_tr, y_mul_tr, X_te, y_mul_te))
    results_mul.append((combo, np.mean(f1s)))


# 6) sort & print top combos
results_bin.sort(key=lambda x: x[1], reverse=True)
results_mul.sort(key=lambda x: x[1], reverse=True)

print("\nTop 10 embedding-pairs for BINARY task (avg wF1):")
for combo, score in results_bin[:10]:
    print(f"  {combo[0]:<8} + {combo[1]:<8} → {score:.4f}")

print("\nTop 10 embedding-pairs for MULTI-CLASS task (avg wF1):")
for combo, score in results_mul[:10]:
    print(f"  {combo[0]:<8} + {combo[1]:<8} → {score:.4f}")


[05/28/25 14:51:09] INFO     Loading data from train_prices (ParquetDataset)...                 ]8;id=571243;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=745383;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from test_prices (ParquetDataset)...                  ]8;id=367872;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=185513;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from fft_train (ParquetDataset)...                    ]8;id=909553;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=414105;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from fft_test (ParquetDataset)...                     ]8;id=435579;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=665267;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from gaf_clip_train (PickleDataset)...                ]8;id=7591;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=645971;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

...

[05/28/25 14:54:24] INFO     Loading data from umap_train (ParquetDataset)...                   ]8;id=841204;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=237971;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from umap_test (ParquetDataset)...                    ]8;id=343266;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=523147;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from wavelet_train (ParquetDataset)...                ]8;id=411823;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=578609;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

                    INFO     Loading data from wavelet_test (ParquetDataset)...                 ]8;id=295268;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=49570;file:///Users/RoverHofman/Desktop/School/thesis/embedding-forecasting/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\


Top 10 embedding-pairs for BINARY task (avg wF1):
  rp_clip  + wavelet  → 0.6730
  pca      + rp_clip  → 0.6693
  gaf_clip + wavelet  → 0.6576
  gaf_clip + pca      → 0.6440
  lle      + wavelet  → 0.6437
  pca      + wavelet  → 0.6386
  mtf_resnet + pca      → 0.6364
  gaf_resnet + pca      → 0.6362
  mtf_clip + wavelet  → 0.6324
  umap     + wavelet  → 0.6324

Top 10 embedding-pairs for MULTI-CLASS task (avg wF1):
  gaf_resnet + rp_clip  → 0.4576
  graph    + umap     → 0.4494
  graph    + rp_clip  → 0.4427
  gaf_dino + gaf_resnet → 0.4415
  rp_clip  + umap     → 0.4385
  pca      + rp_clip  → 0.4379
  mtf_resnet + pca      → 0.4369
  mtf_resnet + wavelet  → 0.4364
  graph    + mtf_resnet → 0.4352
  lle      + pca      → 0.4341
